In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd
import json, datetime

In [2]:
root_path = ''

with open(root_path+'appdata/db_auth.json') as f:
        temp = json.load(f)
        server = temp['server']
        database = temp['database'] 
        username = temp['username'] 
        password = temp['password']

connection_string = 'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)

In [ ]:
query = f"""
SELECT * FROM dbo.supplier
"""

with engine.connect() as conn, conn.begin():  
    sup_list = pd.read_sql(query, conn)

In [ ]:
sup_list[sup_list['SupplierName'].str.startswith('SHAKE')]

In [7]:
query = f"""
SELECT 
    item.ItemLookupCode, 
    sp.SupplierID, 
    dbo.Supplier.SupplierName, 
    dbo.Supplier.Code, 
    item.Description as Description,
    item.Price Price, 
    item.Cost itemCost, 
    sp.Cost spCost,
    dbo.PurchaseOrderEntry.Price recentPrice
FROM 
    dbo.Item item
LEFT JOIN 
    dbo.SupplierList sp ON item.ID=sp.ItemID
LEFT JOIN 
    dbo.Supplier ON sp.SupplierID=dbo.Supplier.ID
LEFT JOIN
    dbo.PurchaseOrderEntry ON item.ID=dbo.PurchaseOrderEntry.ItemID
WHERE 
    item.DepartmentID IN (2,4,6)
    AND sp.SupplierID IN (2123)
    AND item.Inactive = 0
ORDER BY 
    item.SupplierID,
    item.Description;
"""

# query = f"""
# SELECT 
#     item.ItemLookupCode, 
#     sp.SupplierID, 
#     dbo.Supplier.SupplierName, 
#     dbo.Supplier.Code, 
#     item.Description as Description,
#     item.Price Price, 
#     item.Cost itemCost, 
#     sp.Cost spCost
# FROM 
#     dbo.Item item
# LEFT JOIN 
#     dbo.SupplierList sp ON item.ID=sp.ItemID
# LEFT JOIN 
#     dbo.Supplier ON sp.SupplierID=dbo.Supplier.ID
# WHERE 
#     item.DepartmentID IN (2,4,6)
#     AND sp.SupplierID IN (2123)
#     AND item.Inactive = 0
# ORDER BY 
#     item.SupplierID,
#     item.Description;
# """

with engine.connect() as conn, conn.begin():  
    fromPOS = pd.read_sql(query, conn, dtype={'ItemLookupCode': str})

In [4]:
fromPOS['Cost']= fromPOS[['itemCost', 'spCost']].apply(max, axis=1)
fromPOS

,ItemLookupCode,SupplierID,SupplierName,Code,Description,Price,itemCost,spCost,Cost
0,616128305110,2123,SHAKE-N-GO,10107,SIA RT4/CINAM,34.99,25.0,25.0,25.0
1,800181420426,2123,SHAKE-N-GO,10107,3H-BW30 NATURAL,429.99,195.0,195.0,195.0
2,800181420433,2123,SHAKE-N-GO,10107,3H-BW30 NATURAL BK,429.99,195.0,195.0,195.0
3,968252659011,2123,SHAKE-N-GO,10107,B-AFRICA 1,39.99,17.5,17.5,17.5
4,968252659028,2123,SHAKE-N-GO,10107,B-AFRICA 1B,39.99,17.5,17.5,17.5
...,...,...,...,...,...,...,...,...,...
10051,968252791179,2123,SHAKE-N-GO,10107,ZOE F4/27,49.99,26.0,26.0,26.0
10052,968252791186,2123,SHAKE-N-GO,10107,ZOE F4/30,49.99,26.0,26.0,26.0
10053,968252791896,2123,SHAKE-N-GO,10107,ZOE RB35,49.99,26.0,26.0,26.0
10054,968252791797,2123,SHAKE-N-GO,10107,ZOE T27,49.99,26.0,26.0,26.0


In [10]:
sng_inv = pd.read_excel(root_path+'inv_data\SNG_inv.xlsx', dtype={'Barcode':str})

In [13]:
sng_cost = sng_inv.merge(fromPOS[['ItemLookupCode', 'Cost']], how='left', left_on='Barcode', right_on='ItemLookupCode')

In [ ]:
low_price = fromPOS[fromPOS['Price']<fromPOS['Cost']]

In [14]:
fromPOS = fromPOS[['ItemLookupCode', 'Description','Price', 'Cost']]

In [ ]:
fromPOS

In [ ]:
all_amazon = pd.read_csv(root_path+'inv_data/Amazon_All+Listings+Report.txt', sep='\t', dtype={'product-id': str})
all_amazon = all_amazon[['seller-sku', 'asin1', 'listing-id',
       'price', 'product-id-type', 'product-id', 'status']]

In [ ]:
test = fromPOS.merge(all_amazon, how='left', left_on='ItemLookupCode', right_on='product-id')

In [15]:
fromPOS.to_excel('test.xlsx')

In [ ]:
test

In [ ]:
test.dropna()

In [ ]:
dup = test[test.duplicated('ItemLookupCode')]

In [ ]:
test.to_excel('test.xlsx', index=False)